## <font color='Red'>MESSY-CLEAN ROOM CLASSIFIER WITH CNN ON AUGMENTED DATASET</font>

### For further information about the notebook and the details about each step, click at the following link https://github.com/Iron486/Clean_messy_room_classification and check the README.md file.

### Import libraries

In [3]:
import os
import numpy as np
from PIL import Image
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import layers
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import statistics
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import time

### Train the model with a CNN with augmented dataset

In [4]:
size=180
TRAINING_DIR = os.path.join('../input/messy-vs-clean-room/images','train')
train_datagen = ImageDataGenerator(rescale=1/255,
                                  rotation_range=3,
                                  width_shift_range=0.1,
                                  height_shift_range=0.1,
                                  shear_range=0.1,
                                  zoom_range=0.1,
                                  horizontal_flip=True,
                                  fill_mode='nearest')


train_generator = train_datagen.flow_from_directory(TRAINING_DIR,
                                                   batch_size=40,
                                                   class_mode='binary',
                                                   target_size=(size,size))

VALIDATION_DIR = os.path.join('../input/messy-vs-clean-room/images', 'val')
validation_datagen = ImageDataGenerator(rescale=1/255)


validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,
                                                             batch_size=40,
                                                             class_mode='binary',
                                                             target_size=(size,size))



In [5]:
tf.random.set_seed(79)

#model = tf.keras.applications.DenseNet169(include_top=True,weights=None,input_shape=(35,35,3),classes=1)
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='linear', input_shape=(size,size, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3)),#activation='linear'),
    tf.keras.layers.Activation('linear'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='linear'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3),  activation='linear'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='linear'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1200, activation='linear'),
    tf.keras.layers.Dropout(rate=0.15),
    tf.keras.layers.Dense(38, activation='linear'),
    tf.keras.layers.Dropout(rate=0.1),
    tf.keras.layers.Dense(1, activation='sigmoid')
      ])

opt = keras.optimizers.Adam(learning_rate=0.000004)

early_stopping_cb=keras.callbacks.EarlyStopping(patience=210,restore_best_weights=True)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

model.summary()
#pd.set_option('display.max_rows', 10)

history = model.fit_generator(train_generator,
                              epochs=7000,
                              verbose=1,
                              validation_data=validation_generator,callbacks=[early_stopping_cb])

In [8]:
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
#plt.gca().set_xlim(0,29)
plt.gca().set_ylim(0,1)
#model.save('my_keras_model_aug_180_slowlearn.h5')
plt.savefig('/kaggle/working/CNN_dropout_augmented_dataset.png')
loss, accuracy = model.evaluate(validation_generator)

#print accuracy    
print('Accuracy: %f' % (accuracy*100))

### Details about the model

In [9]:
#model=keras.models.load_model('my_keras_model_aug_180_slowlearn.h5') 

#model.summary()

for i, layer in enumerate (model.layers):
    print (i, layer)
    try:
        print ("    ",layer.activation)
    except AttributeError:
        print('   no activation attribute')
#specific info about each layer
for i in range(len(model.layers)):
    print(model.layers[i].get_config())
#info about optimizers
model.optimizer.get_config()      

### Show the entire validation dataset

In [10]:
images=[] 
n_classes=2
for i,filenames in enumerate(os.listdir(VALIDATION_DIR)): 
    dir_fold=os.path.join(VALIDATION_DIR, filenames)
    class_images=[]
    for filename in os.listdir(dir_fold):
        img_path = os.path.join(dir_fold, filename)
        class_images.append(mpimg.imread(img_path))
    images.append(class_images)

#valid for balanced classes
columns = 5
fig,axes=plt.subplots(int(len(images)*len(images[0])/columns),columns,figsize=(30,15))
for i,ind in enumerate(images):
    fig.suptitle('Validation Dataset images',fontsize=47)
    for j, image in enumerate(ind):
        axes[int((i)*(len(ind)/columns)+(int(j/columns))),j % columns].imshow(image)

In [ ]:
len(images[0])

### Predict on test dataset

In [21]:
TEST_DIR = '../input/messy-vs-clean-room/images'
test_datagen = ImageDataGenerator(rescale=1/255)
test_generator = test_datagen.flow_from_directory(TEST_DIR,batch_size=40,
                                                             classes=['test'],
                                                              # don't generate labels
                                                              class_mode=None,
                                                              # don't shuffle
                                                              shuffle=False,
                                                             target_size=(size,size))

In [12]:
files=test_generator.filenames
files    

In [41]:
#test_generator.reset()
pred=np.rint(model.predict(test_generator))
#pred=np.argmax(pred, axis=-1)
predicted_class_indices=np.array(pred).reshape(len(pred))
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
filenames=test_generator.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})
results

#### In this notebook, 90% accuracy on validation and 80% on test were reached. 

#### Training the model with my local GPU using the same model, 90% accuracy was reached for both test and validation datasets. Check out below:

#### https://github.com/Iron486/Clean_messy_room_classification/blob/main/CNN_augmented_dataset_with_dropout.ipynb.

In [42]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
tf.config.list_physical_devices('GPU')